本次以英雄联盟对局胜负预测任务为基础，要求实现决策树算法相关细节，加深对算法的理解，并了解做机器学习任务的大致流程。

### 任务介绍
英雄联盟（League of Legends，LoL）是一个多人在线竞技游戏，由拳头游戏（Riot Games）公司出品。在游戏中，每位玩家控制一位有独特技能的英雄，红蓝两支队伍各有五位玩家进行对战，目标是摧毁对方的基地水晶。水晶有多座防御塔保护，通常需要先摧毁一些防御塔再摧毁水晶。玩家所控制的英雄起初非常弱，需要不断击杀小兵、野怪和对方英雄来获得金币、经验。经验可以提升英雄等级和技能等级，金币可以用来购买装备提升攻击、防御等属性。对战过程中一般没有己方单位在附近的地点是没有视野的，即无法看到对面单位，双方可以通过使用守卫来监视某个地点，洞察对面走向、制定战术。
本数据集来自[Kaggle](https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min)，包含了9879场钻一到大师段位的单双排对局，对局双方几乎是同一水平。每条数据是前10分钟的对局情况，每支队伍有19个特征，红蓝双方共38个特征。这些特征包括英雄击杀、死亡，金钱、经验、等级情况等等。一局游戏一般会持续30至40分钟，但是实际前10分钟的局面很大程度上影响了之后胜负的走向。作为最成功的电子竞技游戏之一，对局数据、选手数据的量化与研究具有重要意义，可以启发游戏将来的发展和改进。

本任务是希望同学们依据注释的要求，对代码中空缺部分进行填写，**完成决策树模型的详细实现**，根据已有的对局前10分钟特征信息，预测最后获胜方是蓝色方还是红色方，了解执行一个**机器学习任务的大致流程**，并**提交代码和实验报告**。第一次作业也是一个机器学习小实验的例子，之后的作业可能不再提供预处理等流程代码，由同学们自己设计实验完成代码编写。

### 导入工具包
pandas是数据分析和处理常用的工具包，非常适合处理行列表格数据。numpy是数学运算工具包，支持高效的矩阵、向量运算。sklearn是机器学习常用工具包，包括了一些已经实现好的简单模型和一些常用数据处理方法、评价指标等函数。

In [2]:
from collections import Counter
import pandas as pd # 数据处理
import numpy as np # 数学运算
from sklearn.model_selection import train_test_split, cross_validate # 划分数据集函数
from sklearn.metrics import accuracy_score # 准确率函数
RANDOM_SEED = 2020 # 固定随机种子

### 读入数据
假设数据文件放在`./data/`目录下，标准的csv文件可以用pandas里的`read_csv()`函数直接读入。文件共有40列，38个特征（红蓝方各19），1个标签列（blueWins），和一个对局标号（gameId）。对局标号不是标签也不是特征，可以舍去。

In [3]:
csv_data = './data/high_diamond_ranked_10min.csv' # 数据路径
data_df = pd.read_csv(csv_data, sep=',') # 读入csv文件为pandas的DataFrame
data_df = data_df.drop(columns='gameId') # 舍去对局标号列

###  数据概览
对于一个机器学习问题，在拿到任务和数据后，首先需要观察数据的情况，比如我们可以通过`.iloc[0]`取出数据的第一行并输出。不难看出每个特征都存成了float64浮点数，该对局蓝色方开局10分钟有小优势。同时也可以发现有些特征列是重复冗余的，比如blueGoldDiff表示蓝色队金币优势，redGoldDiff表示红色方金币优势，这两个特征是完全对称的互为相反数。blueCSPerMin是蓝色方每分钟击杀小兵数，它乘10就是10分钟所有小兵击杀数blueTotalMinionsKilled。在之后的特征处理过程中可以考虑去除这些冗余特征。
另外，pandas有非常方便的`describe()`函数，可以直接通过DataFrame进行调用，可以展示每一列数据的一些统计信息，对数据分布情况有大致了解，比如blueKills蓝色方击杀英雄数在前十分钟的平均数是6.14、方差为2.93，中位数是6，百分之五十以上的对局中该特征在4-8之间，等等。

In [4]:
print(data_df.iloc[0]) # 输出第一行数据
print(data_df.loc[0,'redGoldPerMin'])
print(data_df[0:1])
print(data_df[data_df['blueWardsPlaced']==28])
print('======')
print(data_df.loc[data_df.blueWardsPlaced==28].index)
print('@@@@@@')
print(data_df.iloc[data_df.loc[data_df.blueWardsPlaced==28].index])
print('######')
data_df.describe() # 每列特征的简单统计信息
print('!!!!!!')
vc = data_df['blueWins'].value_counts()
print(vc)


blueWins                            0.0
blueWardsPlaced                    28.0
blueWardsDestroyed                  2.0
blueFirstBlood                      1.0
blueKills                           9.0
blueDeaths                          6.0
blueAssists                        11.0
blueEliteMonsters                   0.0
blueDragons                         0.0
blueHeralds                         0.0
blueTowersDestroyed                 0.0
blueTotalGold                   17210.0
blueAvgLevel                        6.6
blueTotalExperience             17039.0
blueTotalMinionsKilled            195.0
blueTotalJungleMinionsKilled       36.0
blueGoldDiff                      643.0
blueExperienceDiff                 -8.0
blueCSPerMin                       19.5
blueGoldPerMin                   1721.0
redWardsPlaced                     15.0
redWardsDestroyed                   6.0
redFirstBlood                       0.0
redKills                            6.0
redDeaths                           9.0


In [5]:
vc = data_df['blueWardsPlaced'].value_counts()
#print(vc)
#print(vc.max())
print(vc[0:1].index)

Int64Index([16], dtype='int64')


C:\Users\chrischen\AppData\Local\Temp\ipykernel_27416\3896021340.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  print(vc[0:1].index)


### 增删特征
传统的机器学习模型大部分都是基于特征的，因此特征工程是机器学习中非常重要的一步。有时构造一个好的特征比改进一个模型带来的提升更大。这里简单展示一些特征处理的例子。首先，上面提到，特征列中有些特征信息是完全冗余的，会给模型带来不必要的计算量，可以去除。其次，相比于红蓝双方击杀、助攻的绝对值，可能双方击杀英雄的差值更能体现出当前对战的局势。因此，我们可以构造红蓝双方对应特征的差值。数据文件中已有的差值是金币差GoldDiff和经验差ExperienceDiff，实际上每个对应特征都可以构造这样的差值特征。

In [6]:
drop_features = ['blueGoldDiff', 'redGoldDiff', 
                 'blueExperienceDiff', 'redExperienceDiff', 
                 'blueCSPerMin', 'redCSPerMin', 
                 'blueGoldPerMin', 'redGoldPerMin'] # 需要舍去的特征列
df = data_df.drop(columns=drop_features) # 舍去特征列
info_names = [c[3:] for c in df.columns if c.startswith('red')] # 取出要作差值的特征名字（除去red前缀）
for info in info_names: # 对于每个特征名字
    df['br' + info] = df['blue' + info] - df['red' + info] # 构造一个新的特征，由蓝色特征减去红色特征，前缀为br
# 其中FirstBlood为首次击杀最多有一只队伍能获得，brFirstBlood=1为蓝，0为没有产生，-1为红
df = df.drop(columns=['blueFirstBlood', 'redFirstBlood']) # 原有的FirstBlood可删除

### 特征离散化
决策树ID3算法一般是基于离散特征的，本例中存在很多连续的数值特征，例如队伍金币。直接应用该算法每个值当作一个该特征的一个取值可能造成严重的过拟合，因此需要对特征进行离散化，即将一定范围内的值映射成一个值，例如对用户年龄特征，将0-10映射到0，11-18映射到1，19-25映射到2，25-30映射到3，等等类似，然后在决策树构建时使用映射后的值计算信息增益。

***本小节要求实现特征离散化，请补全相关代码***

In [7]:
discrete_df = df.copy() # 先复制一份数据
# 本身取值就很少的特征，brTowersDestroyed数据有些特殊
skip_columns = [
    'blueEliteMonsters','blueDragons','blueHeralds','blueTowersDestroyed',
    'brEliteMonsters','brDragons','brHeralds','brTowersDestroyed',
    'redEliteMonsters','redDragons','redHeralds','redTowersDestroyed',
    'brFirstBlood'
]
for c in df.columns[1:]: # 遍历每一列特征，跳过标签列
    '''
    请离散化每一列特征，即discrete_df[c] = ...

    提示：
    对于有些特征本身取值就很少，可以跳过即 if ... : continue
    对于其他特征，可以使用等区间离散化、等密度离散化或一些其他离散化方法
    可参考使用pandas.cut或qcut
    '''
    if c in skip_columns: continue
    discrete_df[c] = pd.qcut(discrete_df[c], q=4, labels=[0,1,2,3])

discrete_df.describe()

,blueWins,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,redEliteMonsters,redDragons,redHeralds,redTowersDestroyed,brFirstBlood,brEliteMonsters,brDragons,brHeralds,brTowersDestroyed
count,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000
mean,0.499038,0.549954,0.361980,0.187974,0.051422,0.573135,0.413098,0.160036,0.043021,0.009616,-0.023180,-0.051119,0.027938,0.008402
std,0.500024,0.625527,0.480597,0.390712,0.244369,0.626482,0.492415,0.366658,0.216900,1.000004,1.067934,0.878945,0.589292,0.324835
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-2.000000,-1.000000,-1.000000,-2.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,2.000000,1.000000,1.000000,4.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,4.000000


### 数据集准备
构建机器学习模型前要构建训练和测试的数据集。在本例中首先需要分开标签和特征，标签是不能作为模型的输入特征的，就好比作业和试卷答案不能在做题和考试前就告诉学生。测试一个模型在一个任务上的效果至少需要训练集和测试集，训练集用来训练模型的参数，好比学生做作业获得知识，测试集用来测试模型效果，好比期末考试考察学生学习情况。测试集的样本不应该出现在训练集中，否则会造成模型效果估计偏高，好比考试时出的题如果是作业题中出现过的，会造成考试分数不能准确衡量学生的学习情况，估计值偏高。划分训练集和测试集有多种方法，下面首先介绍的是随机取一部分如20%作测试集，剩下作训练集。sklearn提供了相关工具函数`train_test_split`。sklearn的输入输出一般为numpy的array矩阵，需要先将pandas的DataFrame取出为numpy的array矩阵。

In [8]:
all_y = discrete_df['blueWins'].values # 所有标签数据
feature_names = discrete_df.columns[1:] # 所有特征的名称
all_x = discrete_df[feature_names].values # 所有原始特征值，pandas的DataFrame.values取出为numpy的array矩阵

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(all_x, all_y, test_size=0.2, random_state=RANDOM_SEED)
all_y.shape, all_x.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape # 输出数据行列信息

((9879,), (9879, 43), (7903, 43), (1976, 43), (7903,), (1976,))

In [9]:
discrete_df.columns[1:]
#discrete_df.shape
t = feature_names.copy()
print(t == feature_names)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]


###  决策树模型的实现
***本小节要求实现决策树模型，请补全算法代码***

In [10]:
# 定义决策树类
class DecisionTree(object):
    def __init__(self, classes, features, 
                 max_depth=10, min_samples_split=10,
                 impurity_t='entropy'):
        '''
        传入一些可能用到的模型参数，也可能不会用到
        classes表示模型分类总共有几类
        features是每个特征的名字，也方便查询总的共特征数
        max_depth表示构建决策树时的最大深度
        min_samples_split表示构建决策树分裂节点时，如果到达该节点的样本数小于该值则不再分裂
        impurity_t表示计算混杂度（不纯度）的计算方式，例如entropy或gini
        '''  
        self.classes = classes
        self.features = features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.impurity_t = impurity_t
        self.root = None # 定义根节点，未训练时为空
        self.entire_label = None # 方便获取label的子集
        self.entire_feature = None # 方便获取feature的列向量
        
    '''
    请实现决策树算法，使得fit函数和predict函数可以正常调用，跑通之后的测试代码，
    要求之后测试代码输出的准确率大于0.6。
    
    提示：
    可以定义额外一些函数，例如
    impurity()用来计算混杂度
    gain()调用impurity用来计算信息增益
    expand_node()训练时递归函数分裂节点，考虑不同情况
        1. 无需分裂 或 达到分裂阈值
        2. 调用gain()找到最佳分裂特征，递归调用expand_node
        3. 找不到有用的分裂特征
        fit函数调用该函数返回根节点
    traverse_node()预测时遍历节点，考虑不同情况
        1. 已经到达叶节点，则返回分类结果
        2. 该特征取值在训练集中未出现过
        3. 依据特征取值进入相应子节点，递归调用traverse_node
    当然也可以有其他实现方式。

    '''
    
        
    def fit(self, feature, label):
        assert len(self.features) == len(feature[0]) # 输入数据的特征数目应该和模型定义时的特征数目相同
        '''
        训练模型
        feature为二维numpy（n*m）数组，每行表示一个样本，有m个特征
        label为一维numpy（n）数组，表示每个样本的分类标签
        
        提示：一种可能的实现方式为
        self.root = self.expand_node(feature, label, depth=1) # 从根节点开始分裂，模型记录根节点
        '''
        feature_names_fit = self.features
        label = pd.DataFrame(label, columns=['blueWins'])
        self.entire_label = label
        feature = pd.DataFrame(feature, columns=self.features)
        self.entire_feature = feature
        self.root = self.expand_node(feature, label, feature_names_fit) # 从根节点开始分裂，模型记录根节点
        print(self.root)

    
    def predict(self, feature):
        assert len(feature.shape) == 1 or len(feature.shape) == 2 # 只能是1维或2维
        '''
        预测
        输入feature可以是一个一维numpy数组也可以是一个二维numpy数组
        如果是一维numpy（m）数组则是一个样本，包含m个特征，返回一个类别值
        如果是二维numpy（n*m）数组则表示n个样本，每个样本包含m个特征，返回一个numpy一维数组
        
        提示：一种可能的实现方式为
        if len(feature.shape) == 1: # 如果是一个样本
            return self.traverse_node(self.root, feature) # 从根节点开始路由
        return np.array([self.traverse_node(self.root, f) for f in feature]) # 如果是很多个样本
        '''
        if len(feature.shape) == 1: # 如果是一个样本
            return self.traverse_node(self.root, feature) # 从根节点开始路由
        return np.array([self.traverse_node(self.root, f) for f in feature], dtype=object) # 如果是很多个样本 (multiindex不是太理解)


    # 计算混杂度，针对每个特征均需要计算，默认熵为标准
    # dataset: n*m
    # label: n*1
    # 返回混杂度值
    def impurity(self, label, t='entropy'):
        num_labels = len(label)
        label_stat = {}
        for i in label.iloc[:,0]:
            if i not in label_stat.keys():
                label_stat[i] = 0
            label_stat[i] += 1
        impurity = 0.0
        for key in label_stat.keys():
            prob = float(label_stat[key])/num_labels
            impurity -= prob * np.math.log2(prob)
        return impurity

    # 计算信息增益并返回增益最大的特征索引
    # dataset: n*m
    # label: n*1
    # 返回增益最大的特征，最大增益
    # 问题：相同的gain（不是gain=0）也是随机选一个？
    def gain(self, dataset, label, feature_names_fit):
        o_impurity = self.impurity(label) # 计算未分裂混杂度
        max_gain = 0.0
        f_name = ''
        for name in feature_names_fit:
            f_impurity = 0.0 # 特征混杂度
            dc = dataset[name]
            unique_values = set(dc)
            for value in unique_values:
                _label = self.entire_label.iloc[dataset[dc==value].index] # 获取每个特征值对应的label向量
                impurity = self.impurity(_label) # 计算每个特征值的混杂度
                f_impurity += (impurity * float(len(_label)) / len(label)) # 计算所有特征值的加权混杂度
            if (o_impurity - f_impurity) > max_gain:
                max_gain = o_impurity - f_impurity
                f_name = name
        return f_name, max_gain

    # 分裂节点（计算根节点混杂度，计算同当前特征集合中每个特征样本的IG，选取IG最大的特征为下一个节点）
    # feature: n*m
    # label: n*1
    def expand_node(self, feature, label, feature_names_fit, depth=1):
        # 分裂结束条件
        # 1. 所有特征样本有相同输出，即label一样
        vc = label.value_counts()
        if len(vc) == 1:
            return vc.index.tolist()[0][0] # vc.index.tolist()[0] tuple
        # 2. 存在特征样本一样，但label不同的情形（噪声或者可能还有其他特征未考虑到），取label中多数者
        # 以上情形应当在样本数据清洗时处理掉
        # 3. 预剪枝
        if len(feature) <= self.min_samples_split:
            return vc.index.tolist()[0][0]

        f_name, max_gain = self.gain(feature, label, feature_names_fit)
        # print('f_name: ',f_name)
        # print('max_gain: ',max_gain)
        selected_f = feature[f_name]
        unique_values = set(self.entire_feature[f_name])
        # if f_name == 'blueTotalExperience':
        #     print("$$$$$$ feature.shape", feature.shape)
        #     print("$$$$$$", selected_f)
        #     print("$$$$$$", unique_values)
        node = {f_name: {}}
        # 去除增益最大的特征
        feature_names_fit = [name for name in feature_names_fit if name != f_name]
        # 遍历增益最大的特征所有值并递归调用expand_node()
        # 深度优先
        for value in unique_values:
            sub_index = selected_f[selected_f==value].index
            if len(sub_index) == 0:
                # 选择的特征向量（子序列）中不包含实际值，以实际值对应label判定（多数理论）
                ef = self.entire_feature[f_name]
                sl_vc = self.entire_label.loc[ef[ef==value].index].value_counts()
                node[f_name][value] = sl_vc.index.tolist()[0][0]
            else:
                sub_feature = feature.loc[sub_index,feature_names_fit]
                sub_label = label.loc[sub_index]
                node[f_name][value] = self.expand_node(sub_feature, sub_label, feature_names_fit)

        return node

    # 预测特征样本结果
    # root: json对象（没想到合适的存储结构，如果用树的话太繁琐）
    # feature: 特征样本，1*m (ndarray类型)
    def traverse_node(self, root, feature):
        # 离散化样本数据（数据清洗时已完成）
        result_label = ''
        fn = list(root.keys())[0] # 根节点特征名称
        fv_dict = root[fn] # 根节点下字典
        f_index = list(self.features).index(fn) # 根节点的列索引
        for key in fv_dict.keys():
            if feature[f_index] == key:
                if type(fv_dict[key]).__name__ == 'dict':
                    result_label = self.traverse_node(fv_dict[key], feature)
                else:
                    result_label = fv_dict[key]

        return result_label

# 定义决策树模型，传入算法参数
DT = DecisionTree(classes=[0,1], features=feature_names, max_depth=5, min_samples_split=10, impurity_t='gini')

DT.fit(x_train, y_train) # 在训练集上训练
p_test = DT.predict(x_test) # 在测试集上预测，获得预测
# print(p_test) # 输出预测值
test_acc = accuracy_score(np.array(p_test, dtype='int64'), y_test) # 将测试预测值与测试集标签对比获得准确率
print('accuracy: {:.4f}'.format(test_acc)) # 输出准确率

{'brTotalGold': {0: {'brAvgLevel': {0: {'brDeaths': {0: 0, 1: {'blueTotalMinionsKilled': {0: {'blueWardsDestroyed': {0: 0, 1: 0, 2: 1, 3: 0}}, 1: 0, 2: 0, 3: 1}}, 2: {'redTotalExperience': {0: 0, 1: {'redTotalGold': {0: 0, 1: 0, 2: {'redWardsPlaced': {0: 1, 1: 1, 2: 0, 3: 0}}, 3: {'redWardsPlaced': {0: 0, 1: 0, 2: 1, 3: 0}}}}, 2: {'redTotalJungleMinionsKilled': {0: {'redTotalMinionsKilled': {0: 0, 1: 0, 2: 0, 3: 1}}, 1: {'blueAssists': {0: 0, 1: 0, 2: 0, 3: 0}}, 2: {'brWardsPlaced': {0: 0, 1: 0, 2: 0, 3: 0}}, 3: {'blueWardsDestroyed': {0: 0, 1: 0, 2: 0, 3: 0}}}}, 3: {'blueDeaths': {0: {'redTowersDestroyed': {0: 0, 1: 1, 2: 0}}, 1: {'blueWardsPlaced': {0: {'brEliteMonsters': {0: 0, 1: 0, 2: 1, -2: 0, -1: 0}}, 1: {'blueWardsDestroyed': {0: 0, 1: 0, 2: 1, 3: 0}}, 2: {'brTotalMinionsKilled': {0: {'blueWardsDestroyed': {0: 0, 1: 0, 2: 0, 3: 0}}, 1: 1, 2: 1, 3: 1}}, 3: 0}}, 2: {'redWardsPlaced': {0: {'brTotalMinionsKilled': {0: {'blueHeralds': {0: 0, 1: 0}}, 1: 0, 2: 0, 3: 1}}, 1: 0, 2: 0, 3

### 模型调优
第一次模型测试结果可能不够好，可以先检查调试代码是否有bug，再尝试调整参数或者优化计算方法。

### 总结
一个完整的机器学习任务包括：确定任务、数据分析、特征工程、数据集划分、模型设计、模型训练和效果测试、结果分析和调优等多个阶段，本案例以英雄联盟游戏胜负预测任务为例，给出了每个阶段的一些简单例子，帮助大家入门机器学习，希望大家有所收获！